# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
from azureml.core import Workspace,Experiment

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [3]:
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.train.automl import AutoMLConfig

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

auto_ml_url_path='https://raw.githubusercontent.com/Mufumi/Udacity-Capstone-Project/main/Spotify_playlist/spotify_playlist.csv'
auto_ml_ds = TabularDatasetFactory.from_delimited_files(path=auto_ml_url_path)

In [4]:
# TODO: Put your automl settings here
# automl_settings = {}

# TODO: Put your automl config here

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=auto_ml_ds,
    label_column_name="liked",
    n_cross_validations=8)

In [5]:
ws = Workspace.from_config()

# TODO: Submit your experiment
auto_ml_experiment=Experiment(ws,"auto_ml_experiment")

# Submit your automl run
auto_ml_run=auto_ml_experiment.submit(config=automl_config,show_output=True)

No run_configuration provided, running on local with default configuration
Running in the active local environment.


Experiment,Id,Type,Status,Details Page,Docs Page
auto_ml_experiment,AutoML_bd2c4cc9-fa03-455b-b004-c517c96ea72b,automl,Preparing,Link to Azure Machine Learning studio,Link to Documentation


Current status: DatasetEvaluation. Gathering dataset statistics.
Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [6]:
from azureml.widgets import RunDetails

RunDetails(auto_ml_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [7]:
#TODO: Save the best model
import joblib

best_run,best_model = auto_ml_run.get_output()

joblib.dump(value=best_model,filename='auto-ml-best_run.pkl')

['auto-ml-best_run.pkl']

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [15]:
from azureml.core.model import InferenceConfig, Model
from azureml.core import Environment
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.webservice import LocalWebservice

# Register the model to deploy
model_name = best_run.properties['model_name']
description = 'AutoML model for predicting track recommendation'
tags={'area': "music", 'type': "classification"}
model = auto_ml_run.register_model(model_name = model_name, 
                                  description = description, 
                                  tags = tags)

In [16]:
# Combine scoring script & environment in Inference configuration
aml_test_env = Environment(name="project-environment")
inference_config = InferenceConfig(entry_script="score.py",
                                   environment=aml_test_env)

# Set deployment configuration
deployment_config = LocalWebservice.deploy_configuration(port=6789)

# Define the model, inference, & deployment configuration and web service name and location to deploy
service = Model.deploy(workspace = ws,
                       name = "my-web-service",
                       models = [model],
                       inference_config = inference_config,
                       deployment_config = deployment_config)

Generating Docker build context.
Package creation Succeeded
Logging into Docker registry 3977cd3874564d8082e4b29d18b76dee.azurecr.io
Logging into Docker registry 3977cd3874564d8082e4b29d18b76dee.azurecr.io
Building Docker image from Dockerfile...
Step 1/5 : FROM 3977cd3874564d8082e4b29d18b76dee.azurecr.io/azureml/azureml_caedc30a6bda9fb9f54dc89c7082e270
 ---> f955cd5fb5cb
Step 2/5 : COPY azureml-app /var/azureml-app
 ---> e8074fe24437
Step 3/5 : RUN mkdir -p '/var/azureml-app' && echo eyJhY2NvdW50Q29udGV4dCI6eyJzdWJzY3JpcHRpb25JZCI6ImI5NjhmYjM2LWYwNmEtNGM3Ni1hMTVmLWFmYWI2OGFlNzY2NyIsInJlc291cmNlR3JvdXBOYW1lIjoiYW1sLXF1aWNrc3RhcnRzLTE1ODU1MyIsImFjY291bnROYW1lIjoicXVpY2stc3RhcnRzLXdzLTE1ODU1MyIsIndvcmtzcGFjZUlkIjoiMzk3N2NkMzgtNzQ1Ni00ZDgwLTgyZTQtYjI5ZDE4Yjc2ZGVlIn0sIm1vZGVscyI6e30sIm1vZGVsc0luZm8iOnt9fQ== | base64 --decode > /var/azureml-app/model_config_map.json
 ---> Running in 9b89391c2409
 ---> 266f0dae1cf6
Step 4/5 : RUN mv '/var/azureml-app/tmpfnckhxxu.py' /var/azureml-app/main.py


TODO: In the cell below, send a request to the web service you deployed to test it.

In [20]:
import requests
import json

uri = service.scoring_uri
requests.get("http://localhost:6789")
headers = {"Content-Type": "application/json"}
data = {
    
    "danceability":0.724,
    "energy":0.6,
    "key":1,
    "loudness":-6.25,
    "mode":0,
    "speechiness":0.087,
    "acousticness":0.28,
    "instrumentalness":6.83e-05,
    "liveness":0.108,
    "valence":0.201,
    "tempo":164.037
}
data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)

print(response.json())

test is {'danceability': 0.724, 'energy': 0.6, 'key': 1, 'loudness': -6.25, 'mode': 0, 'speechiness': 0.087, 'acousticness': 0.28, 'instrumentalness': 6.83e-05, 'liveness': 0.108, 'valence': 0.201, 'tempo': 164.037}


TODO: In the cell below, print the logs of the web service and delete the service

In [21]:
print(service.get_logs())

2021-09-17T13:53:38,963495688+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2021-09-17T13:53:38,964705385+00:00 - rsyslog/run 
2021-09-17T13:53:38,967305078+00:00 - nginx/run 
2021-09-17T13:53:38,970254370+00:00 - iot-server/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-09-17T13:53:39,065720959+00:00 - iot-server/finish 1 0
2021-09-17T13:53:39,067174461+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 20.1.0
Listening at: http://127.0.0.1:31311 (12)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 41
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2021-09-17 13:53:39,438 | root | INFO | Starting up app insights client
logging socket was found. logging is available.
logging socket was found. logging is available.
2021-09-17 13:53:39,438 | root | INFO | Starting up request id generator
2021-09-17 13:53:39,438 | root | INFO | Star

In [ ]:
service.delete()
model.delete()